In [1]:
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import joblib

# Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('stopwords')


# Cargar los datos
df = pd.read_csv('datos_procesados.csv')



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


In [2]:
# Función de preprocesamiento
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()

    # Eliminar caracteres especiales
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenización
    tokens = word_tokenize(text)

    # Eliminar stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    return ' '.join(tokens)

# Aplicar preprocesamiento
df['processed_text'] = df['Text'].apply(preprocess_text)

# Preparar características (X) y etiquetas (y)
X = df['processed_text']
y = df['IsToxic']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/sunbay/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.10/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.10/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.10/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [17]:

# Definir la función objetivo para Optuna
def objective(trial):
    # Definir el espacio de búsqueda para los hiperparámetros
    c = trial.suggest_loguniform('c', 1e-5, 1e2)
    tfidf_max_features = trial.suggest_int('tfidf_max_features', 1000, 10000)

    # Crear el pipeline con los hiperparámetros sugeridos
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=tfidf_max_features)),
        ('clf', LogisticRegression(C=c, random_state=42))
    ])

    # Realizar validación cruzada
    score = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='f1')

    # Retornar la media del f1-score
    return score.mean()

# Crear un estudio de Optuna
study = optuna.create_study(direction='maximize')

# Optimizar
study.optimize(objective, n_trials=100)

# Imprimir los mejores hiperparámetros y el mejor score
print('Mejores hiperparámetros:', study.best_params)
print('Mejor f1-score:', study.best_value)


[I 2024-11-14 10:05:25,080] A new study created in memory with name: no-name-3f9a7430-a4b8-407f-931f-9c93fc33a35b
/var/folders/qr/kzmym8694wx_f7n4f428rd380000gn/T/ipykernel_14885/1546128899.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  c = trial.suggest_loguniform('c', 1e-5, 1e2)
[W 2024-11-14 10:05:25,150] Trial 0 failed with parameters: {'c': 40.08874066852979, 'tfidf_max_features': 4503} because of the following error: NameError("name 'X_train' is not defined").
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/qr/kzmym8694wx_f7n4f428rd380000gn/T/ipykernel_14885/1546128899.py", line 14, in objective
    score = cross_val_score(pipeline, X_tra

NameError: name 'X_train' is not defined

In [57]:

# Entrenar el modelo final con los mejores hiperparámetros
best_c = study.best_params['c']
best_max_features = study.best_params['tfidf_max_features']

final_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=best_max_features)),
    ('clf', LogisticRegression(C=best_c, random_state=42))
])

final_pipeline.fit(X_train, y_train)

# Evaluar el modelo final
y_pred = final_pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

# Función para clasificar nuevos comentarios
def classify_comment(comment):
    processed = preprocess_text(comment)
    prediction = final_pipeline.predict([processed])
    return "Tóxico" if prediction[0] else "No tóxico"


              precision    recall  f1-score   support

       False       0.66      0.80      0.72        93
        True       0.78      0.64      0.71       107

    accuracy                           0.71       200
   macro avg       0.72      0.72      0.71       200
weighted avg       0.73      0.71      0.71       200



In [1]:
import joblib

# Asumiendo que tu modelo se llama 'modelo'
joblib.dump(final_pipeline, 'modelo_EML_Youtube.joblib')

ModuleNotFoundError: No module named 'joblib'